In [1]:
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import json
import requests 
import time
import random

## Lets look at models! 

In [2]:
#Pull list of models
response = requests.get('http://bigg.ucsd.edu/api/v2/models')
response.json()

{'results': [{'bigg_id': 'e_coli_core',
   'gene_count': 137,
   'metabolite_count': 72,
   'organism': 'Escherichia coli str. K-12 substr. MG1655',
   'reaction_count': 95},
  {'bigg_id': 'iAB_RBC_283',
   'gene_count': 346,
   'metabolite_count': 342,
   'organism': 'Homo sapiens',
   'reaction_count': 469},
  {'bigg_id': 'iAF1260',
   'gene_count': 1261,
   'metabolite_count': 1668,
   'organism': 'Escherichia coli str. K-12 substr. MG1655',
   'reaction_count': 2382},
  {'bigg_id': 'iAF1260b',
   'gene_count': 1261,
   'metabolite_count': 1668,
   'organism': 'Escherichia coli str. K-12 substr. MG1655',
   'reaction_count': 2388},
  {'bigg_id': 'iAF692',
   'gene_count': 692,
   'metabolite_count': 628,
   'organism': 'Methanosarcina barkeri str. Fusaro',
   'reaction_count': 690},
  {'bigg_id': 'iAF987',
   'gene_count': 987,
   'metabolite_count': 1109,
   'organism': 'Geobacter metallireducens GS-15',
   'reaction_count': 1285},
  {'bigg_id': 'iAPECO1_1312',
   'gene_count': 131

In [ ]:
#make a query
response = requests.get('http://bigg.ucsd.edu/api/v2/search?query=iJO1366&search_type=models')
response.json()

# Single Response Example

In [ ]:

model_id='iJO1366'

url2='http://bigg.ucsd.edu/api/v2/models/' + str(model_id) + "/metabolites/"
print (url2)
r2=requests.get(url2)
print (r2.status_code)
if r2.status_code == requests.codes.ok:
    print ("woot woot")
else:
    print ("failed status", r2.status_code, bigg_id)
r2=r2.json()
r2["results"]

## Lets search Bigg!


In [ ]:
#makes folder in directory 
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [ ]:
def search(searchtype, query):
    payload = {'search_type': str(searchtype), 'query': str(query)}
    r=requests.get('http://bigg.ucsd.edu/api/v2/search', params=payload)
    print(r.url, r.status_code)
    return r.json()

In [ ]:
r=search("models", "ijo1366")
print (r.values)
dictionary=r
str(dictionary["results"][0]["bigg_id"])

In [ ]:
cwd=os.getcwd()
cwd

In [ ]:
def download_models(model_list, output_path):
    #iterates through model list searching for matching models
    #if single match, downloads the model as a json at output path
    #Folder : output : subfolders: <same as query name>
    stem="C:\\Users\\zafri\\Documents\\Senior_Design"
    for i in model_list:
        model_search=search("models", str(i))
        print ("searching model..", i)
        if model_search["results_count"]>1:
            print (model_search["results"], "Please refine your search and try again")
        if model_search["results_count"]==0:
            print ("No results were found for", i)
        if model_search["results_count"]==1: 
            model_id=str(model_search["results"][0]["bigg_id"])
            print (model_id)
            cwd=os.getcwd()
            print ("Current Directory:", cwd)
            output_loc=os.path.join(stem, output_path)
            os.chdir(output_loc)
            createFolder(str(i))
            os.chdir(os.path.join(output_loc, i))
            url='http://bigg.ucsd.edu/api/v2/models/' + str(model_id) +'/download'
            url2='http://bigg.ucsd.edu/api/v2/models/'
            print (url)
            r=requests.get(url)
            print (r.status_code)
            with open(model_id+ ".json", "w") as write_file:
                json.dump(r.json(), write_file)
    return r

   

In [ ]:
r=download_models(["ijo1366", "Recon3d", "Recon1", "lactis", "cerevisae"], "Output")

In [ ]:
def download_all_models(model_list, output_path, cutoff):
    stem="C:\\Users\\zafri\\Documents\\Senior_Design"
    for models in model_list:
        model_search=search("models", str(models))
        print ("searching model..", models)
        if model_search["results_count"]>cutoff:
            print (model_search["results"], "# of models found > cuttoff")
        if model_search["results_count"]==0:
            print ("No results were found for", i)
        if model_search["results_count"]>=cutoff: 
            for i in range(cutoff):
                model_id=str(model_search["results"][i]["bigg_id"])
                print (model_id)
                cwd=os.getcwd()
                print ("Current Directory:", cwd)
                output_loc=os.path.join(stem, output_path)
                os.chdir(output_loc)
                createFolder(str(models))
                os.chdir(os.path.join(output_loc, models))
                url='http://bigg.ucsd.edu/api/v2/models/' + str(model_id) +'/download'
                url2='http://bigg.ucsd.edu/api/v2/models/'+str(model_id)
                print (url)
                print (url2)
                r=requests.get(url)
                print (r.status_code)
                with open(model_id +".json", "w") as write_file:
                    json.dump(r.json(), write_file)
    return r

In [ ]:
r=download_all_models(["Recon3d", "Recon1", "cerevisae"], "Output", 2)

In [ ]:
model_id='iJO1366'
url='http://bigg.ucsd.edu/api/v2/models/' + str(model_id) +'/reactions'
r=requests.get(url)
print(url)
r.json()
rstring=r.text
df=pd.read_json(rstring, orient='index')
df

# PULL LISTS FROM MODEL

## Pull Rxn list for Model 

Input: Model ID
Output: PANDAS dataframe with keys (['bigg_id', 'model_bigg_id', 'name', 'organism'])


In [ ]:

def get_rxn_(model_id):
    # for all reactions in given model, pull ec if available 
    start = time.time()
    url='http://bigg.ucsd.edu/api/v2/models/' + str(model_id) +'/reactions'
    r=requests.get(url)
    r=r.json()
    results_list=[]
    if r["results_count"]==0:
        print ("no results for model name, please match model id")
    else:
        for i in r["results"]:
            #print (i)
            results_list.append(i)
    df = pd.DataFrame(results_list)
    return df


In [ ]:
IJO_rxn_list= get_rxn_("iJO1366")
RECON_rxn_list= get_rxn_("RECON1")

In [ ]:
IJO_rxn_list.head()

In [ ]:
RECON_rxn_list.head()

## Pull EC info for each Rxn

In [ ]:

def get_rxn_ec(model_id,t):
    # for all reactions in given model, pull ec if available 
    start = time.time()
    url='http://bigg.ucsd.edu/api/v2/models/' + str(model_id) +'/reactions'
    r=requests.get(url)
    r=r.json()
    results_list=[]
    for i in r["results"]:
        #print (i)
        results_list.append(i)
    df = pd.DataFrame(results_list)

    # now obtain ec numbers 
    #iterate through bigg ids
    count=0
    ec_database_info=[]
    no_database_links=[]
    results=[]  #contains subsystem info
    old_identifiers=[]
    name=[]
    metabolites=[]
    approx_run_time=((len(df["bigg_id"])*t))
    print ("Approx Run Time", approx_run_time/60, "Minutes")
    for bigg_ids in df["bigg_id"]:
        url2='http://bigg.ucsd.edu/api/v2/models/' + str(model_id) + "/reactions/"+ str(bigg_ids)
        print (url2)
        r2=requests.get(url2)
        if r2.status_code == requests.codes.ok:
            print ("Running...", bigg_ids)
            r2=r2.json()
            #DATABASE_LINKS == EC NUMBER/ LINKS DICT
            if len(r2["database_links"])>0:                
                ec_database_info.append([r2["database_links"]])
            else:
                ec_database_info.append("NONE")
           #INSERT REMOVED CODE HERE
            if len(r2["results"])>0:                
                results.append([r2["results"]])
            else:
                subsystem.append("NAN")
            #old_identifiers
            if len(r2["old_identifiers"])>0:                
                old_identifiers.append([r2["old_identifiers"]])
            else:
                old_identifiers.append("NAN")
            #name
            if len(r2["name"])>0:                
                name.append([r2["name"]])
            else:
                name.append("NAN")
            #metabolites
            if len(r2["metabolites"])>0:                
                metabolites.append([r2["metabolites"]])
            else:
                metabolites.append("NAN")
        else:
            print ("failed status", r2.status_code, bigg_ids)
            return (df["bigg_id"], ec_database_info,results,old_identifiers, metabolites)
            break
        time.sleep(t)
        count+=1
        print (count)
        #if count ==100:
         #   break
    #df2 = pd.DataFrame(df["bigg_id"],ec_database_info )
    #df2 = pd.DataFrame(df["bigg_id"], ec_database_info,results,old_identifiers, metabolites)
    
    end = time.time()
    print(end - start)
    return (df["bigg_id"], ec_database_info,results,old_identifiers, metabolites)
        

In [ ]:
ecoli_df_id_list , ecoli_ec_database_info, ecoli_results, ecoli_old_identifiers, ecoli_metabolites=get_rxn_ec("iJO1366", 1)

In [ ]:
recon_df_id_list , recon_ec_database_info, recon_results, recon_old_identifiers, recon_metabolites=get_rxn_ec("RECON1", 1)

In [ ]:
Ecoli_df= pd.DataFrame({"BIGG_ID": ecoli_df_id_list, "EC_Links": ecoli_ec_database_info, "Subsystem/Genes": ecoli_results,
                 "old_identifiers": ecoli_old_identifiers, "Metabolites" : ecoli_metabolites})

In [ ]:
Recon_df= pd.DataFrame({"BIGG_ID": recon_df_id_list, "EC_Links": recon_ec_database_info, "Subsystem/Genes": recon_results,
                 "old_identifiers": recon_old_identifiers, "Metabolites" : recon_metabolites})

In [ ]:
model_id='iJO1366'
bigg_id="12DGR141tipp"

url2='http://bigg.ucsd.edu/api/v2/models/' + str(model_id) + "/reactions/"+ bigg_id
print (url2)
r2=requests.get(url2)
print (r2.status_code)
if r2.status_code == requests.codes.ok:
    print ("woot woot")
else:
    print ("failed status", r2.status_code, bigg_id)
r2=r2.json()
if 'database_links' in r2.keys():
    print (True)
r2.keys()

In [ ]:
for i in [ecoli_df_id_list , ecoli_ec_database_info, ecoli_results, ecoli_old_identifiers, ecoli_metabolites]:
    print (len(i))

In [ ]:
Ecoli_df= pd.DataFrame({"BIGG_ID": ecoli_df_id_list[0:100], "EC_Links": ecoli_ec_database_info, "Subsystem/Genes": ecoli_results,
                 "old_identifiers": ecoli_old_identifiers, "Metabolites" : ecoli_metabolites})

In [ ]:
Ecoli_df

In [ ]:
cwd = os.getcwd()
print ("Current Directory: ", cwd)

In [ ]:
def Dataframe_excel_output(df_name, output_path, Filename):
    os.chdir(output_path)
    writer = pd.ExcelWriter(str(Filename)+'.xlsx', engine='xlsxwriter')
    df_name.to_excel(writer, sheet_name='Sheet2')
    writer.save()
    print ("File name: ", str(Filename)+'.xlsx')

In [ ]:
Dataframe_excel_output(Ecoli_df,'C:\\Users\\zafri\\Documents\\Senior_Design\\Output', "Ecoli_database")

In [ ]:
Dataframe_excel_output(Recon_df,'C:\\Users\\zafri\\Documents\\Senior_Design\\Output', "Recon_database")

In [ ]:
model_id='iJO1366'

url2='http://bigg.ucsd.edu/api/v2/models/' + str(model_id) + "/metabolites/"
print (url2)
r2=requests.get(url2)
print (r2.status_code)
if r2.status_code == requests.codes.ok:
    print ("woot woot")
else:
    print ("failed status", r2.status_code, bigg_id)
r2=r2.json()

In [ ]:
r2["results"]

In [ ]:
def pull_all_met_list(model):
    url2='http://bigg.ucsd.edu/api/v2/models/' + str(model) + "/metabolites/"
    print (url2)
    r2=requests.get(url2)
    print (r2.status_code)
    if r2.status_code == requests.codes.ok:
           print ("woot woot")
    else:
           print ("failed status", r2.status_code, bigg_id)
       
    r2=r2.json()
    mets=r2["results"]
    return mets
        

In [ ]:
IJO_met=pull_all_met_list("iJO1366")
print (len(IJO_met))
RECON1_met=pull_all_met_list("RECON1")
print (len(RECON1_met))
RECON3d_met=pull_all_met_list("RECON3D")
print (len(RECON3d_met))

In [ ]:
IJO_met=pull_all_met_list("iJO1366")

In [ ]:
IJO_unique_metabolites=pd.DataFrame({"IJO1366": IJO_met})
RECON_unique_metabolites=pd.DataFrame({"RECON1": RECON1_met})
        

In [ ]:
def Dataframe_excel_output(df_name, output_path, Filename):
    os.chdir(output_path)
    writer = pd.ExcelWriter(str(Filename)+'.xlsx', engine='xlsxwriter')
    df_name.to_excel(writer, sheet_name='Sheet2')
    writer.save()
    print ("File name: ", str(Filename)+'.xlsx')

In [ ]:
Dataframe_excel_output(IJO_unique_metabolites,'C:\\Users\\zafri\\Documents\\Senior_Design\\Output', "IJO1366_unique_metabolites")

In [ ]:
Dataframe_excel_output(RECON_unique_metabolites,'C:\\Users\\zafri\\Documents\\Senior_Design\\Output', "RECON_unique_metabolites")

# pull SBML files


In [3]:
model_id='iJO1366'

url2='http://bigg.ucsd.edu/static/models/' + str(model_id) + ".xml"
print (url2)
r2=requests.get(url2)
print (r2.status_code)
if r2.status_code == requests.codes.ok:
    print ("woot woot")
else:
    print ("failed status", r2.status_code, bigg_id)


http://bigg.ucsd.edu/static/models/iJO1366.xml
200
woot woot


In [4]:
r2.

<Response [200]>


In [ ]:
cwd=os.getcwd()
sub_files = os.listdir(cwd)
print (sub_files)
